# Import modules

In [1]:
from netCDF4 import Dataset as Dataset
import xarray as xr
import numpy as np
import matplotlib
import glob
from datetime import datetime
from wrf import getvar, ALL_TIMES
import netCDF4 as nc
import glob
from datetime import datetime
import shapefile
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from shapely.geometry import Polygon, Point
import numpy.ma as ma
import scipy
import scipy.stats as stats
from matplotlib import pyplot as plt
import pandas as pd
import pytz

# Load in data

In [3]:
daily_precipitation=xr.open_dataset('/data/PeruGROWS/analysis_data/daily_precip.nc')
lats=xr.open_dataset('/data/PeruGROWS/analysis_data/inner_dom_lats.nc')
lons=xr.open_dataset('/data/PeruGROWS/analysis_data/inner_dom_lats.nc')

# First attempt at clustering

In [5]:
precip_flat=daily_precipitation.stack(z=('south_north','west_east'))